In [ ]:
import glob; import re
import pandas as pd
import numpy as np

In [ ]:
address = ""
all_files = glob.glob(address+'Data_Ny/*.csv')
data_csv = pd.read_csv(all_files[0], index_col=0)

In [ ]:
# Greedy Adjoining Algorithm (GAA)

site_no = len(all_files)

location = [0 for _ in range(site_no)]
for i in range(site_no):
    location[i] = [float(re.findall('\d+.\d+', all_files[i])[0]), float(re.findall('\d+.\d+', all_files[i])[1])]

distance = np.zeros((site_no,site_no))
for i in range(site_no):
    for j in range(site_no):
        distance[i][j] = np.linalg.norm(np.array(location[i]) - np.array(location[j]))

distance_sum = np.zeros((site_no,1))
for i in range(site_no):
    distance_sum[i] = sum(distance[i,:])

site_unselect = list(range(site_no))
site_sort = [0 for _ in range(site_no)]

site_sort[0] = np.argmax(distance_sum)
for i in range(1,site_no):
    site_unselect.remove(site_sort[i-1])
    site_sort[i] = site_unselect[np.argmin(distance[site_sort[i-1],site_unselect])]

In [ ]:
# Data Preprocessing

data_csv.columns = [0]

capacity = [0 for _ in range(site_no)]
for i in range(site_no):
    capacity[i] = int(re.findall('\d+', all_files[i])[5])
    DF = pd.read_csv(all_files[i], index_col=0)
    if i != 0:
        data_csv[i] = DF['Power(MW)']

data_unit = 12 # Time resolution (unit : 5 minute)
data_size = int(len(data_csv)/data_unit)

data = np.zeros((site_no, data_size))
for k in range(site_no):
    for i in range(data_size):
        j = i*data_unit
        data[k][i] = (pd.Series.mean(data_csv[site_sort[k]][j:j+data_unit])/capacity[site_sort[k]])

#pd.DataFrame(data).to_csv(address+'Data_Ny.csv')